<h1><center>Did the 45th President of the United States Tweet it?</center></h1>
<h3><center>by: steven otis</center></h3>

<h2>Data Classes</h2>

In [1]:
import random
class Who:
    TRUMP = 'TRUMP'
    NOT_TRUMP = 'NOT_TRUMP'
class Tweet:
    def __init__(self,text,by):
        self.text = text
        self.by = by
        self.who = self.get_who()
    def get_who(self):
        if  self.by == 'Trump':
            return Who.TRUMP
        elif self.by == 'Not Trump':
            return Who.NOT_TRUMP
class TweetContainer:
    def __init__(self,tweets):
        self.tweets = tweets
    def get_text(self):
        return [x.text for x in self.tweets]
    def get_who(self):
        return [x.who for x in self.tweets]
    def shuffle(self):
        random.shuffle(self.tweets)
        
    

<h2>Load Data</h2>

In [2]:
import pandas as pd


df =pd.read_csv('./data/trumpandrandomtweets.csv')

In [3]:
tweets = []
for row in df.itertuples():
    tweets.append(Tweet(row[1],row[2]))
print('text: ',tweets[0].text)
print('who: ',tweets[0].who)

text:  @ cmedenise Thanks.
who:  TRUMP


<h2>Prep Data</h2>

In [4]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(tweets,test_size=0.33,random_state=42)

train_container = TweetContainer(training)
test_container = TweetContainer(test)

In [5]:
train_container.shuffle()
train_x = train_container.get_text()
train_y = train_container.get_who()

test_container.shuffle()
test_x = test_container.get_text()
test_y = test_container.get_who()

print(train_y.count(Who.TRUMP))
print(train_y.count(Who.NOT_TRUMP))

28383
28292


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vectors[0].toarray())

. @ SbelSbel3555 Sorry, @ piersmorgan is a very smart guy!
[[0. 0. 0. ... 0. 0. 0.]]


<h2>Classification </h2>

<h4>Linear SVM</h4>

In [7]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [ ]:
# optional grid search for parms 
logModel = linear_model.LogisticRegression()

param_grid = [    
    {'penalty' : ['l1', 'l2', 'elasticnet', 'none'],
    'C' : np.logspace(-4, 4, 20),
    'solver' : ['lbfgs','newton-cg','liblinear','sag','saga'],
    'max_iter' : [100, 1000,2500, 5000]
    }
]
clf = GridSearchCV(logModel, param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1)
clf.fit(train_x_vectors,train_y)

In [8]:
# fit model with pre found best prarms.
clf = LogisticRegression(C = 29.763514416313132, max_iter = 1000, penalty = 'l2', solver = 'saga', n_jobs=-1)
clf.fit(train_x_vectors,train_y)


LogisticRegression(C=29.763514416313132, class_weight=None, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=1000, multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=None, solver='saga', tol=0.0001, verbose=0,
                   warm_start=False)

<h2>Evaluation</h2>

In [9]:
print('Mean Accuracy: ',clf.score(test_x_vectors,test_y))

from sklearn.metrics import f1_score
print('F1 Scores: ',f1_score(test_y,clf.predict(test_x_vectors),average=None,labels=[Who.TRUMP,Who.NOT_TRUMP]))


Mean Accuracy:  0.9650725416442772
F1 Scores:  [0.96498222 0.9651624 ]


In [10]:
new_test = vectorizer.transform([test_x[1]])
print(test_x[1])
print(clf.predict(new_test))

forgot what sunshine felty like. I am beet red literally. Bad sunburn. Not fun 
['NOT_TRUMP']


<h2>Saving Model</h2>

In [11]:
#save model
import pickle

Save_folder = './Models'

with open (Save_folder+'/trump_classifier.pkl','wb') as f:
    pickle.dump(clf,f)
with open (Save_folder+'/vectorizer.pkl','wb') as f:
    pickle.dump(vectorizer,f)

<h2>Load Model</h2>

In [12]:
#load model
import pickle
load_folder ='./Models'
with open (load_folder+'/trump_classifier.pkl','rb') as f:
    loaded_clf = pickle.load(f)
#load vectorizer
with open (load_folder+'/vectorizer.pkl','rb') as f:
    loaded_vectorizer = pickle.load(f)

<h2>Predictions</h2>

In [13]:
from ipywidgets import widgets
from IPython.display import clear_output,display,HTML

clf = loaded_clf
vectorizor = loaded_vectorizer

def did_he_say(text,clf,vectorizor):
    v_text = vectorizor.transform([text])
    prediction = clf.predict(v_text)
    return(prediction[0])

def display_prediction(predict):
    if did_he_say(predict.value) == Who.TRUMP:
            display(HTML('<h3><font color="blue">"{}"</font></h2> <h2>That <font color="green">DOES</font> Sounds like <font color="red">TRUMP</font></h2>'.format(predict.value)))

    elif did_he_say(predict.value) == Who.NOT_TRUMP:
        display(HTML('<h3><font color="blue">"{}"</font></h2><h2>Does <font color="red">NOT</font> Sounds like <font color="red">TRUMP</font><h2>'.format(predict.value)))
        
input_text = widgets.Text(description="Tweet")
display(input_text)
input_text.on_submit(display_prediction)

Text(value='', description='Tweet')